### Sentiment Analysis on IMBd movie reviews

Load in libraries

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import TensorDataset, DataLoader
import tensorflow as tf
from torch.autograd import Variable

from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler 

### Load in datasets and create dataloader for NN

In [ ]:
vocab_list = pd.read_csv('imdb.vocab', header = None)

data2 = open("labeledBow.feat", "r", encoding="utf8")

data_size = len(vocab_list)
train = np.zeros(data_size)

train_list = []
train_label = []

cnt = 0

for line in data2:
  line = line.strip()
  indexes = line.split(" ")
  indexes.pop(0)

  train = np.zeros(data_size + 2)
  chance = np.random.uniform(0, 1)

  if chance<=0.075:
    for i in indexes:
      pair = i.split(":")
      idx = int(pair[0])
      val = int(pair[1])

      train[idx] = val

    if cnt<=12499:
      train_label.append([1,0]) #posivite
    else:
      train_label.append([0,1]) #negative
        
    train_list.append(train)

  cnt+=1

data2.close()

In [ ]:
test_list = []
test_label = []

cnt = 0

data2 = open("labeledBow.feat", "r", encoding="utf8")

for line in data2:
  line = line.strip()
  indexes = line.split(" ")
  indexes.pop(0)

  test = np.zeros(data_size)
  chance = np.random.uniform(0, 1)

  if chance<=0.1:
    for i in indexes:
      pair = i.split(":")
      idx = int(pair[0])
      val = int(pair[1])

      test[idx] = val

    if cnt<=12499:
      test_label.append([1,0]) #posivite
    else:
      test_label.append([0,1]) #negative
        
    test_list.append(test)

  cnt+=1

data2.close()

In [ ]:


#train_list = np.array(train_list)
#train_label = np.array(train_label)

test_list = np.array(test_list)
test_label = np.array(test_label)

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 32

'''
train_tensor_x = torch.Tensor(train_list) # transform to torch tensor
train_tensor_y = torch.Tensor(train_label)
train_dataset = TensorDataset(train_tensor_x, train_tensor_y) # create your datset
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
'''

test_tensor_x = torch.Tensor(test_list) # transform to torch tensor
test_tensor_y = torch.Tensor(test_label)
test_dataset = TensorDataset(test_tensor_x, test_tensor_y) # create your datset
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

NameError: ignored

In [ ]:
class MLP1(nn.Module):
    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(89527, 1024)
        self.sig1 = nn.Sigmoid()

        self.fc2 = nn.Linear(1024, 512)
        self.sig2 = nn.Sigmoid()

        self.output = nn.Linear(512, 2)
        self.sig3 = nn.Sigmoid()

    def forward(self, x):
        x = torch.flatten(x, 1)
        
        x = self.fc1(x)
        x = self.sig1(x)

        x = self.fc2(x)
        x = self.sig2(x)

        x = self.output(x)
        x = self.sig3(x)

        return x
  
mlp1 = MLP1()



In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp1.parameters(), lr=0.001)

### Trying to do multiple training sessions of random selection

In [ ]:
%%time

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 32

vocab_list = pd.read_csv('imdb.vocab', header = None)

for train_num in range(3):
  
  f = open("labeledBow.feat", "r", encoding="utf8")

  data_size = len(vocab_list)
  train = np.zeros(data_size)
  train_list = []
  train_label = []

  cnt = 0

  for line in f:
    line = line.strip()
    indexes = line.split(" ")
    indexes.pop(0)

    train = np.zeros(data_size)
    chance = np.random.uniform(0, 1)

    if chance<=0.1: #randomly take 10% of the data set
      for i in indexes:
        pair = i.split(":")
        idx = int(pair[0])
        val = int(pair[1])

        train[idx] = val

      if cnt<=12499: #first half is positive reviews
        train_label.append([1,0]) #posivite
      else:
        train_label.append([0,1]) #negative
          
      train_list.append(train)

    cnt+=1

  f.close()

  train_list = np.array(train_list)

  train_tensor_x = torch.Tensor(train_list) # transform to torch tensor
  train_tensor_y = torch.Tensor(train_label)
  train_dataset = TensorDataset(train_tensor_x, train_tensor_y) # create your datset
  train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

  print("Starting training session: ", train_num+1)

  for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = mlp1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 10 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

print('Finished Training')


Starting training session:  1
[1,    10] loss: 0.714
[1,    20] loss: 0.706
[1,    30] loss: 0.686
[1,    40] loss: 0.682
[1,    50] loss: 0.651
[1,    60] loss: 0.589
[1,    70] loss: 0.549
[2,    10] loss: 0.423
[2,    20] loss: 0.405
[2,    30] loss: 0.385
[2,    40] loss: 0.386
[2,    50] loss: 0.369
[2,    60] loss: 0.376
[2,    70] loss: 0.389
[3,    10] loss: 0.337
[3,    20] loss: 0.327
[3,    30] loss: 0.336
[3,    40] loss: 0.331
[3,    50] loss: 0.337
[3,    60] loss: 0.331
[3,    70] loss: 0.328
[4,    10] loss: 0.318
[4,    20] loss: 0.332
[4,    30] loss: 0.322
[4,    40] loss: 0.322
[4,    50] loss: 0.320
[4,    60] loss: 0.317
[4,    70] loss: 0.328
[5,    10] loss: 0.326
[5,    20] loss: 0.314
[5,    30] loss: 0.320
[5,    40] loss: 0.314
[5,    50] loss: 0.319
[5,    60] loss: 0.326
[5,    70] loss: 0.318
Starting training session:  2
[1,    10] loss: 0.486
[1,    20] loss: 0.443
[1,    30] loss: 0.446
[1,    40] loss: 0.418
[1,    50] loss: 0.439
[1,    60] loss: 0.4

### Saving the model

In [ ]:
print("Model's state_dict:")
for param_tensor in mlp1.state_dict():
    print(param_tensor, "\t", mlp1.state_dict()[param_tensor].size())

torch.save(mlp1.state_dict(), 'NLP_model.pt')

Model's state_dict:
fc1.weight 	 torch.Size([1024, 89527])
fc1.bias 	 torch.Size([1024])
fc2.weight 	 torch.Size([512, 1024])
fc2.bias 	 torch.Size([512])
output.weight 	 torch.Size([2, 512])
output.bias 	 torch.Size([2])


In [ ]:
%%time

for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = mlp1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 10 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 10:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,    10] loss: 0.700
[1,    20] loss: 0.694
[1,    30] loss: 0.681
[1,    40] loss: 0.665
[1,    50] loss: 0.614
[2,    10] loss: 0.500
[2,    20] loss: 0.424
[2,    30] loss: 0.403
[2,    40] loss: 0.380
[2,    50] loss: 0.380
[3,    10] loss: 0.340
[3,    20] loss: 0.333
[3,    30] loss: 0.327
[3,    40] loss: 0.332
[3,    50] loss: 0.325
[4,    10] loss: 0.321
[4,    20] loss: 0.315
[4,    30] loss: 0.320
[4,    40] loss: 0.321
[4,    50] loss: 0.318
[5,    10] loss: 0.317
[5,    20] loss: 0.315
[5,    30] loss: 0.315
[5,    40] loss: 0.314
[5,    50] loss: 0.318
Finished Training
CPU times: user 3min 32s, sys: 2.18 s, total: 3min 34s
Wall time: 3min 34s


In [ ]:
test_review = [[0]]
test_label = [[0]]

cnt = 0

correct = 0
total = 0

data2 = open("labeledBow_test.feat", "r", encoding="utf8")

for line in data2:
  line = line.strip()
  indexes = line.split(" ")
  indexes.pop(0)

  test = np.zeros(data_size)
  chance = np.random.uniform(0, 1)

  for i in indexes:
    pair = i.split(":")
    idx = int(pair[0])
    val = int(pair[1])

    test[idx] = val

  if cnt<=12499:
    test_label[0] = [1,0] #posivite
  else:
    test_label[0] = [0,1] #negative
      
  test_review[0] = test

  cnt+=1

  test_review = np.array(test_review)
  test_label = np.array(test_label)

  # transform to torch tensor
  features = torch.Tensor(test_review) 
  labels = torch.Tensor(test_label)

  with torch.no_grad():
      # calculate outputs by running features through the network
      outputs = mlp1(features)
      # round the ouputs to make decision as whether review is pos or neg
      predicted = torch.round(outputs.data)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

data2.close()

acc = 100*(correct/2)//total
print(f'Accuracy of the network on the test reviews: {100 * (correct/2) // total} %')

Accuracy of the network on the test reviews: 84.0 %


In [ ]:
correct = 0
total = 0


with torch.no_grad():
    for data in test_dataloader:
        review, labels = data
        # calculate outputs by running features through the network
        outputs = mlp1(review)
        # round the ouputs to make decision as whether review is pos or neg
        predicted = torch.round(outputs.data)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

mlp2_acc = 100*(correct/2)//total
print(f'Accuracy of the network on the test reviews: {100 * (correct/2) // total} %')

Accuracy of the network on thetest images: 87.0 %


## References:

[1] PyTorch utils.Data: https://pytorch.org/docs/stable/data.html

[2] PyTorch Training a classifier: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

[3] Pandas library: https://pandas.pydata.org/

[4] Scikit-learn library: https://scikit-learn.org/stable/
